In [87]:
import requests
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [88]:
d1 = open('/home/chirag/Downloads/Stsbenchmark/stsbenchmark/sts-dev.csv')
data = d1.readlines()
print(len(data))

1500


In [89]:
df = []
for i in range(len(data)):
    df.append(data[i].split('\t'))
score = []
sentence1 = []
sentence2 = []
for j in range(len(df)):
    score.append(df[j][4])
    sentence1.append(df[j][5])
    b = df[j][6]
    sentence2.append(b.rstrip("\n"))

In [90]:
dict = list(zip(sentence1, sentence2, score))
df = pd.DataFrame(dict, columns = ['sentence1', 'sentence2', 'score'])

In [91]:
scaler = MinMaxScaler()
df['score'] = scaler.fit_transform(df['score'].values.reshape(-1,1))

In [92]:
df.head()

,sentence1,sentence2,score
0,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,1.00
1,A young child is riding a horse.,A child is riding a horse.,0.95
2,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,1.00
3,A woman is playing the guitar.,A man is playing guitar.,0.48
4,A woman is playing the flute.,A man is playing a flute.,0.55


In [94]:
URL = 'https://salesken-analysis-249709.appspot.com/sentence_similarity_roberta'
rs = []
rob = []
for i in range(len(df)):
    data = {'sentence1': "[\"" + df.sentence1[i] + "\"]",
        'sentence2': "[\"" + df.sentence2[i] + "\"]"}
    r = requests.post(url = URL, data = data) 
#     print(r.json())
    rs.append(r.json())
    rob.append(rs[i][0][0]["roberta_score"])
    print(i)

In [95]:
df["roberta_app_engine"] = rob

In [104]:
from fairseq.models.roberta import RobertaModel
from scipy.stats import pearsonr

In [105]:
roberta = RobertaModel.from_pretrained(
    '/home/chirag/Downloads/checkpoint/',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='/home/chirag/Downloads/checkpoint/ROBERTa-STS-B_STS-B-bin/STS-B-bin'
)

loading archive file /home/chirag/Downloads/checkpoint/
loading archive file /home/chirag/Downloads/checkpoint/ROBERTa-STS-B_STS-B-bin/STS-B-bin
| [input] dictionary: 50265 types


In [ ]:
rb = []
for i in range(len(df)):
    sent1 = df.sentence1[i]
    sent2 = df.sentence2[i]
    tokens = roberta.encode(sent1, sent2)
    features = roberta.extract_features(tokens)
    predictions = roberta.model.classification_heads['sentence_classification_head'](features)
    rb.append(predictions.item())
df["roberta_local"] = r

In [ ]:
df.head()

In [97]:
import numpy as np
x=np.array(df["roberta_app_engine"])
df["roberta_app_engine"] =np.asfarray(x,float)

In [98]:
df.describe()

,score,roberta_app_engine
count,1500.000000,1500.000000
mean,0.472782,0.425463
std,0.300097,0.275280
min,0.000000,-0.097885
25%,0.200000,0.188125
50%,0.510000,0.391519
75%,0.720000,0.642949
max,1.000000,1.029365


In [99]:
scaler = MinMaxScaler()
df['roberta_app_engine'] = scaler.fit_transform(df['roberta_app_engine'].values.reshape(-1,1))
df.describe()

In [100]:
# import psycopg2
# try:
#     connection = psycopg2.connect(user="postgres",
#                                   password="cx6ac54nmgGtLD1y",
#                                   host="35.200.234.61",
#                                   port="5432",
#                                   database="sales")
#     for i in range(len(df)):
#         cursor = connection.cursor()
#         postgres_insert_query = """ UPDATE public.sts_benchmark SET roberta_score = %s, source_='sts_dev' WHERE sentence1 = %s and sentence2 = %s and score=%s"""
#         record_to_insert = (df.roberta_app_engine[i], df.sentence1[i], df.sentence2[i], df.score[i])
#         cursor.execute(postgres_insert_query, record_to_insert)
#         connection.commit()
#         count = cursor.rowcount
#         print (i, "Record inserted successfully into mobile table")
# except (Exception, psycopg2.Error) as error :
#     if(connection):
#         print("Failed to insert record into mobile table", error)
# finally:
#     #closing database connection.
#     if(connection):
#         cursor.close()
#         connection.close()
#         print("PostgreSQL connection is closed")

In [101]:
from scipy.stats import pearsonr
pearson_coef, p_value = pearsonr(df["score"], df["roberta_app_engine"])
print("Pearson Correlation Coefficient: ", pearson_coef, "and a P-value of:", p_value)

Pearson Correlation Coefficient:  0.7953677432367878 and a P-value of: 0.0


In [102]:
# y_true = [1,0,0,0,1,1]
# y_pred = [1,1,0,1,0,1]
# tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
